# 東森新聞雲爬蟲練習
## 練習從東森新聞雲網站中，爬取新聞摘要及詳細內容。
## 學習利用Selenium模擬人為操作，更新動態網頁後爬取新聞內容。

In [8]:
import requests
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

### 以財經新聞為例，先連結到財經新聞網頁，複製其URL。

In [9]:
ETtoday_url = "https://www.ettoday.net/news/focus/%E8%B2%A1%E7%B6%93/"  #財經新聞

### 用Selenium打開一個瀏覽器，連結到網站。然後模擬下拉網頁的動作，讓網頁完成更新，再處理後續動作。

In [10]:
browser = webdriver.Chrome()
browser.get(ETtoday_url)  # 打開瀏覽器並連到東森新聞雲網頁

SCROLL_PAUSE_TIME = 1

#
# 以下是用Selenium模擬下拉網頁動作，讓網頁更新
#
last_height = browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
while True:
    '''
    Your code here
    
    提示：可參考以下的Stack Overflow: 
    https://stackoverflow.com/questions/48850974/selenium-scroll-to-end-of-page-indynamically-loading-webpage/48851166
    '''
    # 模擬網頁下拉
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # 等待一段時間讓網頁更新
    time.sleep(SCROLL_PAUSE_TIME)

    # 檢視目前網頁底端
    new_height = browser.execute_script("return document.body.scrollHeight")
    
    if new_height == last_height:
        print("到達頁面底端")
        break
    last_height = new_height
    print("網頁更新中...")

網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
網頁更新中...
到達頁面底端


### 到這裡網頁已經更新完畢，所有的新聞都已經出現在網頁上。接下來做爬取收集新聞的動作。

In [11]:
#
# 這裡先建立一個函數，其功能是連到外部連結，並爬取新聞內容。
#
def getNewsDetailContent(link_url):
    resp = requests.get(link_url)
    resp.encoding = 'utf-8'
    #print(resp.text)

    soup = BeautifulSoup(resp.text, 'lxml')
    news_content = soup.find("div", attrs={'class':'story'}).find_all("p")
    for p in news_content:
        """
        .string屬性說明：
        (1) 若當前tag節點底下沒有其他tag子節點，會直接抓取內容(返回"NavigableString")
        (2) 若當前tag節點底下只有唯一的一個tag子節點，也會直接抓取tag子節點的內容(返回"NavigableString")
        (3) 但若當前tag節點底下還有很多個tag子節點，.string就無法判斷，(返回"None")
        """
        if ((p.string) is not None):
            print(p.string)

### 解析HTML並萃取新聞摘要，若有外部的連結，再連到外部連結並把詳細新聞內容抓取下來。

In [12]:
# 爬取網頁內容，解析後萃取新聞摘要
html = browser.page_source
soup = BeautifulSoup(html, "lxml")
all_news = soup.find("div", attrs={'class':'block block_1 infinite_scroll'})

news_block = all_news.find_all('div', attrs={'class':'piece clearfix'})

for i, news_item in enumerate(news_block):
    print("----------------------------------------------------------------------")
    news_body = news_item.find('h3')
    print("\n[%d] %s\n" % (i, news_body.a.string))
    
    # 
    # 連到外部連結，擷取詳細新聞內容
    #
    externalLink = "https://www.ettoday.net" + news_body.a["href"]
    getNewsDetailContent(externalLink)


----------------------------------------------------------------------

[0] 武漢肺炎疫情蔓延　國銀在陸據點啟動3大防疫機制備戰

▲武漢肺炎疫情蔓延尚未見遏止之勢，全球各地陸續傳出確診病例。（圖／達志影像／美聯社）
記者紀佳妘／台北報導
武漢肺炎疫情持續擴散，在大陸設有分行、子行的國銀今（7）日表示，配合當地政府相關規定開工，且各據點也啟動緊急應變措施，包括分組上班、追蹤員工身體健康情形等，也落實各項防疫措施。
富邦金控已公布「富邦金融控股股份有限公司暨子公司因應嚴重特殊傳染性肺炎之緊急應變機制」，提供全體同仁遵循，並全面啟動相關防疫機制，目前針對辦公及營業場所採取3大防疫措施。
首先，公司各棟辦公大樓全面對所有進出同仁、訪客進行量體溫及使用酒精消毒；第二，各子公司營業處第一線同仁，全面配戴口罩；第三，強化宣導請同仁以肥皂勤洗手、戴口罩，全面防疫，並落實同仁一旦有確診病例或不適症狀立即確實通報。
富邦金100%持股的子公司富邦華一銀行在大陸設有26個營業據點，到1月底為止，富邦華一台籍員工一共有151人。富邦華一銀行表示，全力配合做好疫情防控工作，開工時間依照各地政府相關規定執行，目前大多數分行恢復營業時間預計為2月10日。
富邦華一銀行指出，各分行也將遵循上述辦公及營業場所防疫措施，並且每日對辦公及營業場所、客戶接待區、辦公機具等進行消毒，同時配置防疫相關的消毒液、洗手液。另在疫情防控期間，富邦華一銀行採用線上軟體開會辦公，並暫緩員工各類出差。
在大陸擁有上海、廣州、廈門、深圳共5家分支機構的中國信託銀行表示，大陸各分行是依當地銀保監局、政府及勞動部門規定，通知員工分別於2月3日及2月10日開工。如當地監管單位另有防疫規定，或因往來交通受阻等特殊考量，得採自主隔離、在家工作。在自主隔離期間，公司正常給薪。
中國信託銀行也提及，目前大陸各分行採行分組上班、彈性工時，到班員工提供醫療口罩、乾洗酒精、每日量測體溫等防衛措施，並強化行舍辦公營業場所衛生管理，每日進行線上問卷調查以即時掌握員工疫情情況，管控可能的疫情風險。
----------------------------------------------------------------------

[1] 台積電去年專利申請首度突破

▲ 新台幣兌美元今天開盤貶值6.8分。（圖／路透社）
記者李瑞瑾／台北報導
台北外匯市場新台幣兌美元匯率今（7）日開盤貶值6.8分、以30.15元兌1美元開出。
新台幣兌美元昨（6）日盤中一度再現29字頭，但仍收在30.082兌1美元；今天開盤則貶值6.8分，以30.15元兌1美元開出。
新台幣開盤後收斂跌勢，截至上午9點09分，新台幣來到30.1元兌1美元，貶值1.8分。
----------------------------------------------------------------------

[13] 台股下跌逾百點！不受美股創高激勵　台股早盤跌破11700點

▲ 台股。（圖／記者湯興漢攝）
記者李瑞瑾／台北報導
中國大陸對美方釋出降稅利多，加上美國財報與經濟數據優於預期，美股三大指數齊刷歷史新高紀錄。台股卻不受美股激勵，在昨日大漲逾百點後拉回整理，今（7）日開盤下跌36點、以11712點開出。
截至上午9點04分，台股跌勢擴大，加權指數來到11,641點、下跌108點。
電子三雄中，台積電（2330）以330元開出、下跌0.75%；鴻海（2317）開在平盤83.6元；股王大立光（3008）以4780元開出、下跌1.34%。
中國大陸宣布調降對美商品關稅、美國強勁財報與經濟數據激勵，加上武漢肺炎對金融市場的影響漸小，美股道瓊、那指、標普同步刷新歷史新高紀錄。
美股道瓊指數上漲88.92點，或0.3%，收29,379.77點；那斯達克指數上漲63.47點，或0.67%，收9,572.15點；標普 500 指數上漲10.81點，或0.32%，收3,345.5點；費城半導體指數上漲0.12點，或0.01%，收1909.64點。
----------------------------------------------------------------------

[14] 賀！捷運環狀線通車　新埔民生站香榭京宴全新落成

圖、文／富比士提供
尤其是離站區僅三分鐘步行距離的「香榭京宴」，明顯地成為捷運通車最大得利者。除了既有的「新埔站」貫穿北市心臟的藍線大動脈，現在再加上360°包抄雙北市的環線系統「新埔民生站」，城市生活最重要的效率與自由就一手在握了，這還沒算上緊鄰的江翠國中與便利的江翠生活圈。
至於「香榭京宴」看不到的內涵，那就更厲害了；

▲依據產創條例規範，企業去年開始投資智慧機械、5G相關，可抵減營所稅。（圖／路透、示意圖）
記者吳靜君／台北報導
政府為推動智慧機械產業的政策，優化產業結構達成智慧升級轉型，並鼓勵多元創新應用，縮短數位落差，訂定《產業創新條例》第10條之一，對於公司或有限合夥事業，自去(2019)年度開始，投資於智慧機械及第五代行動通訊系統（5G）相關硬體、軟體、技術或技術服務的支出，應於辦理當年度營所稅結算申報期間開始前4個月起至申報期間截止日內申請，可以適用抵減應納營利事業所得稅額。
KPMG安侯建業聯合會計師事務所稅務部營運長張芷表示，依據產業創新條例，自2019年1月1日起至2021年12月31日止，投資於自行使用全新智慧機械，或自2019年1月1日起至2022年12月31日止投資於導入5G相關全新硬體、軟體、技術或技術服務之支出，只要依規定提出投資計畫，並經各中央目的事業主管機關專案核准者，就得申請適用投資抵減營所稅稅額。
張芷進一步說明，只要公司或有限合夥事業去年的智慧機械投資或導入第五代行動通訊系統(5G)之相關支出，年度總金額合計達100萬元以上、10億元以下者，得選擇兩種方式抵減：
投資抵減兩種方式　企業可擇優扣抵
一、於支出金額5%限度內，抵減當年度應納營利事業所得稅額
二、於支出金額3%限度內，自當年度起3年內抵減各年度應納營利事業所得稅額。
一經選定不得變更，抵減稅額以不超過當年度應納營利事業所得稅額30%為限。若有合併適用其他投資抵減，如研發支出投資抵減等項目，其合計得抵減總額則不能超過當年度應納營利事業所得稅額50%，但若依其他法律規定當年度為最後抵減年度且抵減金額不受限制者，則適用其他法律之投資抵減稅額不受50%的限制。
舉例說明，例如甲公司去年度投資於智慧機械支出3000萬元，甲公司108年度申報營利事業所得稅應納稅額為500萬元，如沒有其他投資抵減項目，甲公司可能選擇第一種5%的抵減方式，抵減之稅額為150萬元（3000萬元×5%=150萬元）也沒有超過應納營利事業所得稅額30%。
但假如甲公司應納稅額為200萬元，甲公司可能就要選擇第二種3%的抵減方式，因甲公司2019年度最多可抵減60萬元(200萬元X30%=60萬元)；投資可抵減的90萬元(3000萬元X3%=90萬元)，但當年度未抵完30萬元部分還可供以後兩年度抵減。
投資抵減申請方式　改

▲武漢肺炎疫情延燒，中國許多城市已經封城。（圖／路透社）
記者楊絡懸／台北報導
武漢肺炎疫情持續升溫，連帶打亂及影響到眾多國人近期出國差旅行程。行政院陸委會已宣布中國旅遊警示燈號調升為「紅色」，但新安東京海上產險6日宣布，均持續提供所有旅遊平安險及旅遊不便險等相關商品投保服務，包含中國的旅遊綜合險。
▲多數民眾戴口罩防疫。（圖／路透社）
▲新型冠狀病毒。（圖／路透社）
新安東京海上產險提醒民眾，出國前應盡早規劃投保足額的旅平險及旅遊不便險等保險，尤其目前全球聞「疫」色變之下，為旅遊或出差行程多添一份心安的保障。
----------------------------------------------------------------------

[37] 影／防疫概念股還有空間？　分析師建議勿追高

武漢肺炎疫情持續延燒，有人認為疫情高峰尚未來臨，因此觀望防疫概念股是否能持續搶進。在《雲端最有錢》節目中，來賓們直言表示，防疫概念股就是你買不到的一定會漲，買得到的趕快賣，因為代表它一定是弱勢股。
財金系教授段昌文拿出2003年SARS期間大家追股票的異常報酬來看，那些防疫股大概只漲了19天，假如現在想搶進，手腳真的要夠快，不然實在機會渺茫。
假如回頭看這幾天大家狂追的個股，像是美德醫療（9103）、桓大（1325）、康那香（9919）、毛寶（1732）、花仙子（1730），段昌文表示，如果要從這些防疫股裡面挑選績優股的話，要先看營業毛利率，最好是超過30%，ROE（股東權益報酬率）大於15%，不過以這標準來挑防疫概念股，可能會選不到，因此除非手腳快狠準，不然就不要追了。
至於該如何設停損點，資深分析師王兆立提出三大點，分別是季線及半年線下彎、本益比（PE）高於20倍（大盤18.76倍）、股價淨值比（PB）高於2倍（大盤1.73倍）。因為這種股票未來縱使漲了，也是彈不高。
----------------------------------------------------------------------

[38] 財訊／台商30年》兩岸半導體纏鬥30年　台灣勝出迭起

文／林宏達
美中全球爭霸，科技，是雙方誰也不可能放棄的舞台；誰擁有了科技的領導權，誰就將在經濟、軍事、教育上全面領先。而半導體，更是科技競爭的重中之重。
台灣半導體崛起　中國看在眼裡　傾

▲ 武漢肺炎讓醫療基金成市場焦點。（示意圖／路透）
記者李瑞瑾／綜合報導
保德信全球醫療生化基金經理人江宜虔表示，目前多家國際藥廠主動提供協助，將抗病毒藥物輸出至疫區，如AbbVie(艾伯維藥廠), Johnson & Johnson(嬌生公司)等公司，有提供相關抗疫情藥物。根據AbbVie(艾伯維藥廠)抗愛滋病毒藥品Kaletra曾在2004年實驗結果顯示，能有效對抗SARS病毒，其他如Johnson & Johnson、Merck & Co.(默克藥廠)、Gilead Science(吉利德)也分別提供相關藥物給疫區。
此外，統計1992至2016美國總統大選年，S&P500醫療產業股價相對S&P500指數表現好換參半，選前6個月相對表現持平或微幅落後，但選後6個月表現明顯超越整體股市。江宜虔認為，長期而言，醫療類股將在財報報佳音加持下，後市展望趨勢仍有可為。
----------------------------------------------------------------------

[50] 敏成配合新型冠狀病毒防疫　24小時不停機增產「醫療用口罩濾材」

▲醫療用口罩。（圖／資料畫面）
記者姚惠茹／台北報導
因應近期爆發的新型冠狀病毒疫情，台灣口罩需求大增，敏成（4431）今（6）日宣布配合政府防疫，已經從開工日就增產熔噴技術不織布，也就是醫療用口罩所需的濾材，並將產能火力全開，24小時不停機生產，全力支援防疫所需。
敏成董事長古思明表示，有鑑於春節假期中疫情持續擴大，敏成已經在開工當日就把產能火力全開，24小時不停機加緊趕工，以台灣民眾健康及國家安全為優先，全力配合政府需求生產口罩濾材，為社會盡一份心力。
敏成是成立近30年的熔噴不織布廠商，具有卓越的熔噴技術與穩定性高製程經驗，採用最佳等級的熔噴過濾材原料，以及與世界接軌的全套自主檢驗設備，產製空氣、水等各式濾材與保溫棉等上游材料，符合國際大廠之規範。
敏成表示，目前歐、美及台灣的一線醫材大廠皆是敏成的客戶，現有口罩濾材訂單已排程到五月份，同時敏成也將過濾材料研發不斷提升，已完成可以吸附並殺死細菌的濾材，讓原料量產到位，並可依市場需求提供適當等級的濾材。
敏成指出，近年來社會大眾對於預防空汙、過敏、流感，以及易傳染的病毒日漸重視，因衛生因素使口罩成為日常必需品，並在這次新型冠狀病毒爆發後

▲eBay。（圖／達志影像／美聯社）
記者陳心怡／外電報導
《華爾街日報》日前報導，紐約證券交易所大股東、美國洲際交易所集團（Intercontinental Exchange，ICE）已向線上拍賣及購物網站eBay發出收購要約。洲際交易所集團表示，已經接觸eBay，探索可能為兩家公司的股東創造價值的潛在機會。
洲際交易所過去就和eBay進行過收購接觸，但兩家公司尚沒有進入正式談判階段，也無法保證 eBay會同意進行該項交易。不過假如雙方真的達成協議，考慮到eBay市值超過280億美元，加上洲際交易所可能會支付的收購溢價，很可能會讓eBay的估值超過300億美元（約新台幣9千億元）。
據了解，洲際交易所主要是對eBay的核心拍賣業務有興趣，而不是eBay有意出售的分類廣告業務。洲際交易所可能是想把eBay電商網站上，將買家和賣家配對起來的技術收歸旗下。
美國時間本週三，eBay股票收盤價報37.12美元，下跌0.28美元。跌幅為0.76%。
----------------------------------------------------------------------

[64] 小英總統視察口罩新產線　經長：派員駐點在工廠「不分日夜衝產量！」

▲總統蔡英文(圖右)昨（5）日下午視察口罩新產線，經濟部長沈榮津（前排左一）陪同解說，政府斥資2億佈建的 62條口罩產線將陸續上線。（圖／總統府提供）
記者林淑慧／台北報導
武漢肺炎疫情延燒，口罩實名制今（6）日上路，蔡英文總統昨日親赴台灣康匠公司視察口罩新產線，陪同的經濟部長沈榮津表示，為提升產能，經濟部已派員直接駐點在工廠，日夜趕工「全力衝產量」。
依據中央流行疫情指揮中心統計，台灣目前有10名嚴重特殊傳染性肺炎（武漢肺炎）患者，10名個案狀況穩定，且有1人採檢都是陰性，可望近日出院，另9名患者也都是輕症，並赴中港澳旅遊回台者，都要進行居家檢疫 14 天，國內口罩需求大增。
經濟部表示，感謝全國許許多多本土口罩工廠，從年假時開始就配合政府提前開工。像是昨天蔡總統參訪的台灣康匠，陳董事長打趣說，他年節休假時，都在接經濟部的電話，因此年初二工廠就開工生產。
「好幾家（口罩）廠商也都一樣，陸續提早開工，也配合轉為全天候不分日夜，都在生產的模式。」隨行的經濟部官員說，為了促成政府民間一條心，經濟部相關同仁春節沒有休

武漢肺炎疫情延燒，中國大陸許多城市封城，工廠也都關閉，而位於湖北的電子廠首當其衝，成為嚴重受災區。在《雲端最有錢》節目中，來賓就點出台廠受害名單，提醒投資人留意。
王兆立以定穎（6251）為例，這間公司共兩個廠區，分別在崑山（營收占比70%）及湖北黃石（營收占比30%），在台灣則是完全沒有廠，把它的股價拿出來看，就會發現這幾天股價非常弱，因此手上有這類型股票的人還是減碼為安。
大家對於目前疫情都感到非常恐慌，但王兆立就提到，美國十年期公債殖利率，尤其是兩年期、三年期、五年期，跌破歷史新低，這代表目前資金往債市流動，假如資金不夠多的情況，前往債市流，股市必崩盤，但美股道瓊、費半、那斯達克指數都在半年線，甚至季線之上，所以全球資金是寬鬆的，大家不需要過度恐慌。
----------------------------------------------------------------------

[79] 友達宣布以每股57元　公開收購凌華科技普通股

▲友達財務長曾煜智。（圖／記者姚惠茹攝）
記者姚惠茹／台北報導
面板廠友達（2409）今（5）日晚間召開重大訊息說明會，由財務長曾煜智說明董事會決議通過，公開收購工業電腦大廠凌華（6166），以每股新台幣57元收購凌華普通股5%～30%。以凌華今日收盤價47.05計算，溢價21.1%，以60天平均股價計算，溢價率為20.5%。
曾煜智強調，凌華作為全球工業電腦領導廠商，在工業與商業應用的許多垂直場域，均已展現領先市場的邊緣運算（edge computing）相關的產品技術與解決方案的競爭力，並在既有工業電腦的基礎上，凌華面對客戶對人工智能應用的需求，提出可擴充性的軟硬體平台整合方案。
展望5G時代來臨，曾煜智表示，友達期望借重凌華在工業電腦領域累積多年的經驗，加上友達做為人機介面的優質面板產品，期望雙方締結策略夥伴促成優勢互補，共同掌握多元領域客戶從自動化到智能化的數位轉型需求。
----------------------------------------------------------------------

[80] 台指期開高走低、收盤漲8點　分析師：近期要快節奏操作、疫情過後再做長線

▲台指期2月5日走勢。（圖／取自期交所）
記者余弦妙／台北報導
台指期今（5）早受美股激勵一開盤上漲69點，來到

▲ 武漢肺炎疫情升溫，央行對受影響產業之票信從寬處理。（示意圖／路透）
記者李瑞瑾／台北報導
因應武漢肺炎疫情升高，央行表示，為協助受本次事件影響票款清償能力之產業，已於今（5）日函請台灣票據交換所依相關規定予以從寬處理。
央行指出，前述受影響業者，可向中央目的事業主管機關（經濟部或交通部等）申請紓困，由該主管機關表明同意輔導，函請央行轉交票交所處理。
給予申請紓困業者「六個月」暫緩通報為拒絕往來戶之寬延期間，在上述期間內，暫不因發生存款不足退票通報為拒絕往來戶；而於上述期間內，將退票辦理清償贖回、提存備付或重提付訖之註記者，該退票紀錄不對外提供查詢。
----------------------------------------------------------------------

[92] 年後轉職「鐵飯碗」複試名單底加　國營事業職缺「起薪36K起跳！」

▲經濟部今（5）日公佈中油、台電、台糖及台水等所屬事業新進人員複試名單。(圖／記者湯興漢攝）
記者林淑慧／台北報導
年後轉職潮蠢動，來搶鐵飯碗。經濟部今（5）日公告國營事業新進職員甄試複試名單；官員指出，台電、台水、台糖與台灣中油等國營事業新進職員甄試，今年預計錄取1440名員額，最終錄取者將可獲36K至39K起薪。
受到一例一休及退休潮影響，經濟部所屬國營事業大舉徵才，經濟部人事處去年7月公布108年度專科以上層級新進職員徵才訊息，徵才單位包托台電、中油、台糖、台水等各國營事業，並於去年11月24日舉行初試。
經濟部官員指出，進入複試名單今天在經濟部公布欄、甄試網站及各國營事業網站同步公告，本次考試初試成績採線上查詢，試務處將以電子郵件及簡訊通知應考人自行至甄試網站查閱成績。
另外，應考人如欲申請成績複查，請於109年2月12日前（郵戳為憑），依甄試簡章規定程序以書面方式申請。
經濟部表示，經中油、台電、台糖及台水公司因業務需要提報，共計增列221名員額，增額後本次甄試錄取總人數為1,440名，以本次招考實際到考人數19,927人計算，錄取率將由6.12%提升至7.23%。
----------------------------------------------------------------------

[93] 11、12月期發票中獎　幸運兒花15元買果菜汁中200萬

▲有民眾在超